In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/test.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
train['date'] = pd.to_datetime(train['date'], format = '%Y-%m-%d')

test = pd.read_csv(file_content_stream_2)
test['date'] = pd.to_datetime(test['date'], format = '%Y-%m-%d')

# Basic Feature Engineering 

In [2]:
train['weekday'] = train['date'].dt.dayofweek
test['weekday'] = test['date'].dt.dayofweek

train['month'] = train['date'].dt.month_name()
test['month'] = test['date'].dt.month_name()

# Basic Prediction using medians

In [17]:
train_means = train.groupby(['country', 'store', 'product', 'month', 'weekday'])['num_sold'].median().to_dict()
test['num_sold'] = test.set_index(['country', 'store', 'product', 'month', 'weekday']).index.map(train_means.get)
test.head()

,row_id,date,country,store,product,weekday,month,num_sold
0,70128,2021-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,4,January,437.0
1,70129,2021-01-01,Belgium,KaggleMart,Kaggle Getting Started,4,January,311.0
2,70130,2021-01-01,Belgium,KaggleMart,Kaggle Recipe Book,4,January,279.0
3,70131,2021-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,4,January,444.0
4,70132,2021-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,4,January,154.0


In [18]:
test['num_sold'] = np.where(test['country'] == 'Poland', test['num_sold'] + 98, test['num_sold'])

In [19]:
test[['row_id', 'num_sold']].to_csv('submission_median_9.csv',index = False)